In [1]:
import typing
import openmdao.api as om
import openmdao.core.system as oms

from standard_evaluator import get_interface, show_structure, create_problem, get_state, set_state, load_state, save_state
from standard_evaluator import load_assembly, save_assembly, convert_om_var, get_opt_problem, set_opt_problem
from standard_evaluator import Variable, FloatVariable, IntVariable, ArrayVariable, OptProblem

As a first step fo this demonstration we generate an OpenMDAO assembly with wo sub-groups, one of which represents an aero analyses with two components. The first component is a stand-in for a geometry builder, and the second component is a stand-in for a Computational Fluid Dynamics component. There is a link between those two components which can be thought of as a large, multi-dimensional grid. In this example we are just using a 1-D array, but this can be thought of as a large three dimensional grid.

In [2]:
grid_size = 200

class Sub1(om.Group):
    def setup(self):
        self.add_subsystem('z1', om.ExecComp("y = a + b + c + d",
                                            ),
                           promotes_inputs=[
                               'a',
                               ('b', 'my_alias'),
                               'd',
                           ])

        self.add_subsystem('z2', om.ExecComp("v = a + b + c + d",
                                            ),
                           promotes_inputs=['c', 'd'],
                           promotes_outputs=['*'])
        self.connect('z1.y', 'c')

class Aero(om.Group):
    def setup(self):
        self.add_subsystem('geometry', om.ExecComp([f"grid = (q + z *l)*ones({grid_size})", f"bla = outer(ones({grid_size}), ones({grid_size}))"], 
                    grid={'tags': 'internal', 'shape': (grid_size)},
                    bla ={'tags': 'internal', 'shape': (grid_size, grid_size)}
                    ),
                    promotes_inputs=['q', 'z', 'l'])
        self.add_subsystem('cfd', om.ExecComp(f"drag = (inner(grid, ones({grid_size})) + r * k)/1000", 
                    grid={'tags': 'internal', 'shape_by_conn': True},
                    drag={'units': 'N'}),
                    promotes_inputs=['r', 'k'],
                    promotes_outputs=['drag'])
        self.connect('geometry.grid', 'cfd.grid')


class Intermediate3(om.Group):
    def setup(self):
        self.add_subsystem('sub1', Sub1(),
            promotes_inputs=['a'])

        self.add_subsystem('aero', Aero())
        self.connect('sub1.v', 'aero.k')

prob4 = om.Problem(model=Intermediate3())

# Define an optimization problem
prob4.model.add_design_var('a', lower=-50, upper=50, scaler= .1, adder=30.)
prob4.model.add_design_var('sub1.my_alias', lower=-50, upper=50)
prob4.model.add_objective('aero.drag')
prob4.model.add_constraint('sub1.v', lower=0, upper=10.)
prob4.model.add_constraint('sub1.z1.y', lower=-32.32, upper=1829.)
#prob4.model.add_constraint('aero.geometry.bla', lower=-3., upper=45.)
#prob4.model.add_constraint('aero.geometry.grid', lower=0, upper=10.)

prob4.setup()
prob4.final_setup()

prob4.run_model()
_ = prob4.model.list_inputs(shape=True)
_ = prob4.model.list_outputs(shape=True)

14 Input(s) in 'model'

varname     val                   shape   prom_name    
----------  --------------------  ------  -------------
sub1
  z1
    a       [1.]                  (1,)    a            
    b       [1.]                  (1,)    sub1.my_alias
    c       [1.]                  (1,)    sub1.z1.c    
    d       [1.]                  (1,)    sub1.d       
  z2
    a       [1.]                  (1,)    sub1.z2.a    
    b       [1.]                  (1,)    sub1.z2.b    
    c       [4.]                  (1,)    sub1.c       
    d       [1.]                  (1,)    sub1.d       
aero
  geometry
    l       [1.]                  (1,)    aero.l       
    q       [1.]                  (1,)    aero.q       
    z       [1.]                  (1,)    aero.z       
  cfd
    grid    |28.28427125|         (200,)  aero.cfd.grid
    k       [7.]                  (1,)    aero.k       
    r       [1.]                  (1,)    aero.r       


5 Explicit Output(s) in 'model'

varname 

We can now take this OpenMDAO assembly and create a Pydantic description of this assembly using the `get_interface` method. This description can be easily saved to a JSON file, and we can also load the information from a JSON file. We also get the state of the OpenMDAO problem which will be saved in a different, binary HDF5 formatted file.

In this example we actually create a new OpenMDAO model from the information, which we call `new_prob4`.

In [3]:
info = get_interface(prob4.model)
state = get_state(prob4, info)
new_prob4 = create_problem(info, state)
_ = new_prob4.model.list_inputs(shape=True)

** This is a group, special handling needed for sub1 in group 
Adding z1 component
Building Equation class z1
Adding z2 component
Building Equation class z2
** This is a group, special handling needed for aero in group 
Adding geometry component
Building Equation class geometry
Adding cfd component
Building Equation class cfd
14 Input(s) in 'model'

varname     val                   shape   prom_name    
----------  --------------------  ------  -------------
sub1
  z1
    a       [1.]                  (1,)    a            
    b       [1.]                  (1,)    sub1.my_alias
    c       [1.]                  (1,)    sub1.z1.c    
    d       [1.]                  (1,)    sub1.d       
  z2
    a       [1.]                  (1,)    sub1.z2.a    
    b       [1.]                  (1,)    sub1.z2.b    
    c       [4.]                  (1,)    sub1.c       
    d       [1.]                  (1,)    sub1.d       
aero
  geometry
    l       [1.]                  (1,)    aero.l       
 

In [4]:
state

{'aero.cfd.grid': array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 'aero.k': array([7.]),
 'aero.r': array([1.]),
 'aero.l': array([1.]),
 'aero.q': ar

In [5]:
# Saving the structure of the assembly and the state in files
save_assembly(prob4, assembly_name='assembly.json', state_name='state.h5')

# Now loading the structure and state from these files
new_problem = load_assembly(assembly_name='assembly.json', state_name='state.h5')
_ = new_problem.model.list_inputs(shape=True, units=True)

** This is a group, special handling needed for sub1 in group 
Adding z1 component
Building Equation class z1
Adding z2 component
Building Equation class z2
** This is a group, special handling needed for aero in group 
Adding geometry component
Building Equation class geometry
Translating set
Translating set
Adding cfd component
Building Equation class cfd
Translating set
14 Input(s) in 'model'

varname     val                   units  shape   prom_name    
----------  --------------------  -----  ------  -------------
sub1
  z1
    a       [1.]                  None   (1,)    a            
    b       [1.]                  None   (1,)    sub1.my_alias
    c       [1.]                  None   (1,)    sub1.z1.c    
    d       [1.]                  None   (1,)    sub1.d       
  z2
    a       [1.]                  None   (1,)    sub1.z2.a    
    b       [1.]                  None   (1,)    sub1.z2.b    
    c       [4.]                  None   (1,)    sub1.c       
    d       [1.]  

We can now save the N2 diagram for both the original and the newly created OpenMDAO model.

In [6]:
om.n2(prob4, 'replace.html', display_in_notebook=False)
om.n2(new_prob4, 'replace_after.html', display_in_notebook=False)

# Replacing a group with a Surrogate or simplified model
In the next step we creata a new OpenMDAO component that we will want to use to replace the complex aero group in the original model with. In this example since we have two simple equations we can replace those two equations with a single equation where we substitute the first equation into the second equation. Note that because we used a simple way of creating the one dimensional grid and using the grid in the second formula we can calculate what the inner product of two vectors of length 20 with all ones in them is, which is 20.

We then take this component and add it to an OpenMDAO `Problem` and run `setup()` to fully instantiate the problem.

Note that this step could easily be replaced with a process where we take the original aero group, instantiate it as an OpenMDAO problem, run a Design of Experiment against it, and build a surrogate model which we can then expose as an OpenMDAO component. This is not part of the contract, but Boeing has some internal tools that demonstrate this process.

In [7]:
class Surrogate(om.Group):
    def setup(self):
        self.add_subsystem('new', 
                om.ExecComp("drag = ((q + z *l)*20 + r * k)/1000",
                drag={'units': 'N'}),
                promotes_inputs=['q', 'z', 'l', 'r', 'k'],
                promotes_outputs=['drag'])



prob_replace = om.Problem(model=Surrogate())
prob_replace.setup()

In the next step we get the Pydantic representation of the new surrogate using the `get_interface` method from the standard evaluator. We give it the same name as the component we are trying to replace.

In [8]:
surrogate_info = get_interface(prob_replace.model)
surrogate_info.name = 'aero'
#show_structure(surrogate_info)

We can now replace the original group with the surrogate component / group in the Pydantic description.

In [9]:
info.components['aero'] = surrogate_info

Create a new OpenMDAO problem from the assembly information where the original 'aero' group is replaced by a simplified group / component.

In [10]:
new_prob5 = create_problem(info)

** This is a group, special handling needed for sub1 in group 
Adding z1 component
Building Equation class z1
Adding z2 component
Building Equation class z2
** This is a group, special handling needed for aero in group 
Adding new component
Building Equation class new


We can now save the N2 diagram of the new OpenMDAO Problem.

In [11]:
om.n2(new_prob5, 'replace_after_surrogate.html', display_in_notebook=False)

Finally, we are setting values for all inputs to the two different assemblies, and run both of them. You will notice that the calculated drag is the same for the two calculations.

In [12]:
prob4.set_val('a', 34.)
prob4.set_val('sub1.my_alias', 5)
prob4.set_val('sub1.z1.c', 24.)
prob4.set_val('sub1.d', 3.)
prob4.set_val('sub1.c', 12.)
prob4.set_val('aero.l', 400.)
prob4.set_val('aero.q', 5.)
prob4.set_val('aero.r', 6.)
prob4.set_val('aero.z', 7.)
prob4.run_model()
_ = prob4.model.list_outputs(shape=True, units=True)

5 Explicit Output(s) in 'model'

varname     val                  units  shape       prom_name         
----------  -------------------  -----  ----------  ------------------
sub1
  z1
    y       [66.]                None   (1,)        sub1.z1.y         
  z2
    v       [71.]                None   (1,)        sub1.v            
aero
  geometry
    bla     |200.0|              None   (200, 200)  aero.geometry.bla 
    grid    |39668.69042457|     None   (200,)      aero.geometry.grid
  cfd
    drag    [561.426]            N      (1,)        aero.drag         


0 Implicit Output(s) in 'model'




In [13]:
# We can get a Python dictionary with the current state of the problem using the `get_state` method
local_dict = get_state(prob4, info)

# We can also save the state using the `save_state` method. This saves the state in the industry standard hdf5 file format
filename = 'data.h5'

save_state(prob4, info, filename)


# Load the state from the HDF5 file using the load_state helper method. Note that we need to capture the info from the new 
# instance so that variables and responses that were part of the old components are not seen anymore.
new_info = get_interface(new_prob5.model)
load_state(new_prob5, new_info, filename)

# Note that we could also set the state using the dictionary that was created by the `get_state` method by using
# the `set_state` method.


In [14]:
#new_prob5.run_model()
_ = new_prob5.model.list_inputs(shape=True, units=True)
_ = new_prob5.model.list_outputs(shape=True, units=True)

13 Input(s) in 'model'

varname  val     units  shape  prom_name    
-------  ------  -----  -----  -------------
sub1
  z1
    a    [34.]   None   (1,)   a            
    b    [5.]    None   (1,)   sub1.my_alias
    c    [24.]   None   (1,)   sub1.z1.c    
    d    [1.]    None   (1,)   sub1.d       
  z2
    a    [1.]    None   (1,)   sub1.z2.a    
    b    [1.]    None   (1,)   sub1.z2.b    
    c    [66.]   None   (1,)   sub1.c       
    d    [1.]    None   (1,)   sub1.d       
aero
  new
    k    [71.]   None   (1,)   aero.k       
    l    [400.]  None   (1,)   aero.l       
    q    [5.]    None   (1,)   aero.q       
    r    [6.]    None   (1,)   aero.r       
    z    [7.]    None   (1,)   aero.z       


3 Explicit Output(s) in 'model'

varname   val        units  shape  prom_name
--------  ---------  -----  -----  ---------
sub1
  z1
    y     [66.]      None   (1,)   sub1.z1.y
  z2
    v     [71.]      None   (1,)   sub1.v   
aero
  new
    drag  [561.426]  N      (1,)  

In [15]:
opt_problem = get_opt_problem(prob4)
opt_problem

OptProblem(name='opt_problem', class_type='OptProblem', variables=[FloatVariable(name='a', default=None, bounds=(-50.0, 50.0), shift=30.0, scale=0.1, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'), FloatVariable(name='sub1.my_alias', default=None, bounds=(-50.0, 50.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float')], responses=[FloatVariable(name='sub1.v', default=None, bounds=(0.0, 10.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'), FloatVariable(name='sub1.z1.y', default=None, bounds=(-32.32, 1829.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'), FloatVariable(name='aero.drag', default=None, bounds=(-inf, inf), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float')], objectives=['aero.drag'], constr

In [16]:
opt_problem.variables

[FloatVariable(name='a', default=None, bounds=(-50.0, 50.0), shift=30.0, scale=0.1, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'),
 FloatVariable(name='sub1.my_alias', default=None, bounds=(-50.0, 50.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float')]

In [17]:
clean_prob = create_problem(get_interface(prob4.model), opt_problem=opt_problem)
#set_opt_problem(clean_prob, opt_problem)

_ = clean_prob.list_driver_vars(show_promoted_name=True, desvar_opts=['lower', 'upper', 'adder', 'scaler'],
                cons_opts=['lower', 'upper', 'adder', 'scaler'],
                objs_opts=['adder', 'scaler'])

** This is a group, special handling needed for sub1 in group 
Adding z1 component
Building Equation class z1
Adding z2 component
Building Equation class z2
** This is a group, special handling needed for aero in group 
Adding geometry component
Building Equation class geometry
Adding cfd component
Building Equation class cfd
----------------
Design Variables
----------------
name           val    size  lower  upper  adder  scaler  
-------------  -----  ----  -----  -----  -----  ------ 
a              [3.1]  1     -2.0   8.0    30.0   0.1     
sub1.my_alias  [1.]   1     -50.0  50.0   None   None    

-----------
Constraints
-----------
name       val   size  lower   upper   adder  scaler  
---------  ----  ----  ------  ------  -----  ------ 
sub1.v     [1.]  1     0.0     10.0    None   None    
sub1.z1.y  [1.]  1     -32.32  1829.0  None   None    

----------
Objectives
----------
name       val   size  adder  scaler  
---------  ----  ----  -----  ------ 
aero.drag  [1.]  1     

In [18]:
import json
optproblem_name = 'optproblem1.json'
# Convert and write JSON object to file
with open(optproblem_name, "w") as outfile:
    json.dump(opt_problem.model_dump(exclude_unset=True), outfile, indent=2, skipkeys=True)


In [19]:
import json_numpy
with open(optproblem_name, "r") as infile:
    info_dict = json_numpy.load(infile)
    new_opt = OptProblem.model_validate(info_dict)
new_opt

OptProblem(name='opt_problem', class_type='OptProblem', variables=[FloatVariable(name='a', default=None, bounds=(-50.0, 50.0), shift=30.0, scale=0.1, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'), FloatVariable(name='sub1.my_alias', default=None, bounds=(-50.0, 50.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float')], responses=[FloatVariable(name='sub1.v', default=None, bounds=(0.0, 10.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'), FloatVariable(name='sub1.z1.y', default=None, bounds=(-32.32, 1829.0), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float'), FloatVariable(name='aero.drag', default=None, bounds=(-inf, inf), shift=0.0, scale=1.0, units=None, description='', options={'parallel_deriv_color': None}, class_type='float')], objectives=['aero.drag'], constr

In [20]:
new_opt == opt_problem

True